<a href="https://colab.research.google.com/github/shwets1503/Sales_Prediction/blob/master/Predicting_customer_satisfaction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [338]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [339]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt

In [340]:
orders_df = pd.read_csv('../content/drive/My Drive/sales_data2/olist_orders_dataset.csv', engine='python')
order_items = pd.read_csv('../content/drive/My Drive/sales_data2/olist_order_items_dataset.csv', engine='python')
order_reviews = pd.read_csv('../content/drive/My Drive/sales_data2/olist_order_reviews_dataset.csv', engine='python')
products = pd.read_csv('../content/drive/My Drive/sales_data3/olist_products_dataset.csv', engine='python')
product_category = pd.read_csv('../content/drive/My Drive/sales_data3/product_category_name_translation.csv', engine='python')
customer = pd.read_csv('../content/drive/My Drive/sales_data/customers_dataset.csv', engine='python')

In [341]:
categories = pd.merge(products, product_category,how ='left', left_on ='product_category_name', right_index = True, suffixes=(None, '_eng')).drop('product_category_name', axis=1)

In [342]:
orders = pd.merge(order_reviews, order_items, how='left', on='order_id')
orders = pd.merge(orders, orders_df, how='left', on='order_id')
orders = pd.merge(orders, customer, how='left', on='customer_id')
orders = pd.merge(orders, categories, how='left', on='product_id')

In [343]:
# converting to datetime
orders['order_purchase_timestamp'] = pd.to_datetime(orders.order_purchase_timestamp)
orders['order_approved_at'] = pd.to_datetime(orders.order_approved_at).dt.date  
orders['order_estimated_delivery_date'] = pd.to_datetime(orders.order_estimated_delivery_date).dt.date  
orders['order_delivered_customer_date'] = pd.to_datetime(orders.order_delivered_customer_date).dt.date  

# Defining the problem
Our problem is to find a way to estimate, based on data about the product and order, what will be the customer review score.
# The hypothesis
Our main hypothesis is that the product and how the order was fulfilled might influence the customer review score. Keep in mind that each feature we create is a new hypothesis we are testing.

#Designing an Experiment
To answer that question we must implement collect data from each order up to delivery phase. With that, we should implement a model that estimates what will be the score given by the customer at the review phase.

In [344]:
orders = orders[['order_status', 'price',
                 'freight_value', 'order_item_id', 
                 'order_purchase_timestamp', 'order_approved_at', 'order_estimated_delivery_date', 
                 'order_delivered_customer_date', 'customer_state', 
                 'product_category_name_eng', 'product_name_lenght', 'product_description_lenght', 
                 'product_photos_qty', 'review_score']]

In [345]:
# We keep the same proportion of classes
orders['review_score'].value_counts() / len(orders['review_score'])

5    0.558265
4    0.187686
1    0.134461
3    0.084110
2    0.035478
Name: review_score, dtype: float64

In [346]:
# Simple Split
from sklearn.model_selection import train_test_split

# split
train_set, test_set = train_test_split(orders, test_size=0.2, random_state=42)
test_set['review_score'].value_counts() / len(test_set['review_score'])

5    0.559202
4    0.185714
1    0.138650
3    0.082121
2    0.034312
Name: review_score, dtype: float64

In [347]:
# Stratified Split
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(orders, orders['review_score']):
    strat_train_set = orders.loc[train_index]
    strat_test_set = orders.loc[test_index]

In [348]:
strat_train_set['review_score'].value_counts() / len(strat_train_set['review_score'])

5    0.558260
4    0.187686
1    0.134465
3    0.084115
2    0.035473
Name: review_score, dtype: float64

In [349]:
strat_train_set.dropna(axis=0,how='any', inplace=True)

By doing a stratified split we keep the same proportion between classes. This split better represent the original data and will possibli reduce any bias.



# Feature Engineering

In [350]:
orders_features = strat_train_set.drop('review_score', axis=1)
orders_labels = strat_train_set['review_score'].copy()

# Creating a Custom Transformer for FeatEng
We need to guarantee that we are apply exactly the same transformation to new/unseen data. To do that we will create custom transformers using scikit-learn BaseEstimator.

This first custom transformer will do the feature engineering that we just described earlier.

In [351]:
corr_matrix = strat_train_set.corr()
corr_matrix['review_score'].sort_values(ascending=False)

review_score                  1.000000
product_photos_qty            0.022280
product_description_lenght    0.013805
price                        -0.000592
product_name_lenght          -0.013109
freight_value                -0.035872
order_item_id                -0.140872
Name: review_score, dtype: float64

In [352]:
# To consider Brazilian calendar and hollidays
!pip install workalendar
from workalendar.america import Brazil
cal = Brazil()

In [353]:
from sklearn.base import BaseEstimator, TransformerMixin
class AttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass    
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        df = X.copy()
        
        # Calculate the estimated delivery time and actual delivery time in working days. 
        # This would allow us to exclude hollidays that could influence delivery times.
        # If the order_delivered_customer_date is null, it returns 0.
        df['wd_estimated_delivery_time'] = df.apply(lambda x: cal.get_working_days_delta(x.order_approved_at, 
                                                                                      x.order_estimated_delivery_date), axis=1)
        df['wd_actual_delivery_time'] = df.apply(lambda x: cal.get_working_days_delta(x.order_approved_at, 
                                                                                   x.order_delivered_customer_date), axis=1)

        # Calculate the time between the actual and estimated delivery date. If negative was delivered early, if positive was delivered late.
        df['wd_delivery_time_delta'] = df.wd_actual_delivery_time - df.wd_estimated_delivery_time


        # Calculate the time between the actual and estimated delivery date. If negative was delivered early, if positive was delivered late.
        df['is_late'] = df.order_delivered_customer_date > df.order_estimated_delivery_date
        
        # Calculate the average product value.
        df['average_product_value'] = df.price / df.order_item_id

        # Calculate the total order value
        df['total_order_value'] = df.price + df.freight_value
        
        # Calculate the order freight ratio.
        df['order_freight_ratio'] = df.freight_value / df.price
        
        # Calculate the order freight ratio.
        df['purchase_dayofweek'] = df.order_purchase_timestamp.dt.dayofweek
                       
        # With that we can remove the timestamps from the dataset
        cols2drop = ['order_purchase_timestamp', 'order_approved_at', 'order_estimated_delivery_date', 
                     'order_delivered_customer_date']
        df.drop(cols2drop, axis=1, inplace=True)
        
        return df

In [354]:
# Executing the estimator we just created
attr_adder = AttributesAdder()
feat_eng = attr_adder.transform(strat_train_set)
feat_eng.head(3)

,order_status,price,freight_value,order_item_id,customer_state,product_category_name_eng,product_name_lenght,product_description_lenght,product_photos_qty,review_score,wd_estimated_delivery_time,wd_actual_delivery_time,wd_delivery_time_delta,is_late,average_product_value,total_order_value,order_freight_ratio,purchase_dayofweek
32719,delivered,39.9,18.23,1.0,MG,computers_accessories,55.0,197.0,1.0,4,19,7,-12,False,39.9,58.13,0.456892,4
43987,delivered,22.5,15.10,1.0,RJ,housewares,51.0,444.0,1.0,5,15,10,-5,False,22.5,37.60,0.671111,0
2976,delivered,69.9,24.71,1.0,RS,furniture_decor,44.0,903.0,6.0,5,22,10,-12,False,69.9,94.61,0.353505,0


In [355]:
corr_matrix = feat_eng.corr()
corr_matrix['review_score'].sort_values(ascending=False)

review_score                  1.000000
product_photos_qty            0.022280
average_product_value         0.014869
product_description_lenght    0.013805
price                        -0.000592
total_order_value            -0.003569
purchase_dayofweek           -0.009580
product_name_lenght          -0.013109
freight_value                -0.035872
order_freight_ratio          -0.038214
wd_estimated_delivery_time   -0.062350
order_item_id                -0.140872
wd_delivery_time_delta       -0.229488
wd_actual_delivery_time      -0.307135
is_late                      -0.356554
Name: review_score, dtype: float64

In [356]:
feat_eng.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 87421 entries, 32719 to 56558
Data columns (total 18 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   order_status                87421 non-null  object 
 1   price                       87421 non-null  float64
 2   freight_value               87421 non-null  float64
 3   order_item_id               87421 non-null  float64
 4   customer_state              87421 non-null  object 
 5   product_category_name_eng   87421 non-null  object 
 6   product_name_lenght         87421 non-null  float64
 7   product_description_lenght  87421 non-null  float64
 8   product_photos_qty          87421 non-null  float64
 9   review_score                87421 non-null  int64  
 10  wd_estimated_delivery_time  87421 non-null  int64  
 11  wd_actual_delivery_time     87421 non-null  int64  
 12  wd_delivery_time_delta      87421 non-null  int64  
 13  is_late                    

# Dealing with Categorical and Numerical Attributes

In [331]:
pip install --upgrade category_encoders

Requirement already up-to-date: category_encoders in /usr/local/lib/python3.6/dist-packages (2.2.2)


In [362]:
# selecting the numerical and text attributes
cat_attribs = ['order_status', 'customer_state', 'product_category_name_eng']
num_attribs = orders_features.drop(cat_attribs, axis=1).columns

In [363]:
import category_encoders as ce
# create an object of the OneHotEncoder
OHE = ce.OneHotEncoder(cols=cat_attribs,use_cat_names=True)
# encode the categorical variables
orders_features = OHE.fit_transform(orders_features)

In [364]:
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        return X[self.attribute_names]

In [365]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# for now we wont work with categorical data. Planning to add it on next releases
num_pipeline = Pipeline([('selector', DataFrameSelector(num_attribs)),
                         ('attribs_adder', AttributesAdder()),
                         ('std_scaller', StandardScaler())
                        ])

In [373]:
# lets see how the resulting data looks like
orders_features_prepared = num_pipeline.fit_transform(orders_features)
orders_features_prepared

array([[-0.44131459, -0.11008851, -0.27982497, ..., -0.43407026,
         0.40466955,  0.63861471],
       [-0.53735833, -0.30902296, -0.27982497, ..., -0.54316711,
         1.03639195, -1.39637208],
       [-0.27572195,  0.30176298, -0.27982497, ..., -0.24021478,
         0.09978501, -1.39637208],
       ...,
       [-0.44081781, -0.39736764, -0.27982497, ..., -0.45761137,
         0.0683217 ,  0.63861471],
       [-0.17029463, -0.40181666, -0.27982497, ..., -0.19754319,
        -0.49073367,  1.6561081 ],
       [ 0.33145109,  0.16511473, -0.27982497, ...,  0.33290236,
        -0.57287814, -1.39637208]])

In [367]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(orders_features_prepared, orders_labels)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [368]:
some_data = orders_features.iloc[:8]
some_labels = orders_labels.iloc[:8]
some_data_prepared = num_pipeline.transform(some_data)

In [369]:
print('Predicted: {} \n Labels: {}'.format(list(lin_reg.predict(some_data_prepared)), list(some_labels.values)))

Predicted: [4.200990438393185, 4.164893735056262, 4.220587093283143, 4.35703562563107, 4.278321832870888, 4.35741398908125, 2.1552175818279866, 4.170959230377667] 
 Labels: [4, 5, 5, 4, 4, 5, 5, 5]


In [370]:
from sklearn.metrics import mean_squared_error

predictions = lin_reg.predict(orders_features_prepared)
lin_mse = mean_squared_error(orders_labels, predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

1.2409979983355306

In [371]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor()
forest_reg.fit(orders_features_prepared, orders_labels)

predictions = forest_reg.predict(orders_features_prepared)
forest_mse = mean_squared_error(orders_labels, predictions)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

0.4621176525516894

In [372]:
print('Predicted: {} \n Labels: {}'.format(list(forest_reg.predict(some_data_prepared)), list(some_labels.values)))

Predicted: [4.09, 4.68, 4.33, 4.2, 4.12, 4.79, 4.65, 4.85] 
 Labels: [4, 5, 5, 4, 4, 5, 5, 5]


Note: Nan values dropped for now but can be imputed. Categorical values changed to one hot encoded. Classification instead of regression starting from logistic regression then others.
Using cross validation set later and grid search. Ending with pipeline to predict.